In [3]:
import os
from datetime import datetime, timedelta
import time
import pandas as pd
from keep_run import keep_alive

from datetime import datetime
import pymongo
from pymongo import MongoClient
import requests
from requests.auth import HTTPDigestAuth
from ipify import get_ip

from flask import Flask
from threading import Thread

app=Flask('')
@app.route('/')

def home():
  return "Its alive"

def run():
  app.run(host='0.0.0.0', port=8080)

def keep_alive():
  t=Thread(target = run)
  t.start()

def cnx(mongod_connect):
    client = MongoClient(mongod_connect)
    db = client.dbt 
    ctweet = db.clx
    #ctweet.drop()
    ctweet.create_index([("id", pymongo.ASCENDING)],unique = True)
    return ctweet

def set_ip(atlas_group_id, atlas_api_key_public, atlas_api_key_private, deleteAfterDate):
    resp = requests.post( "https://cloud.mongodb.com/api/atlas/v1.0/groups/{atlas_group_id}/accessList".
    format(atlas_group_id=atlas_group_id),auth=HTTPDigestAuth(atlas_api_key_public, atlas_api_key_private),
        json=[{'ipAddress': get_ip(), 
              'deleteAfterDate': deleteAfterDate,
              'comment': 'replit :'+str(datetime.now().strftime("%Y-%m-%dT%H"))}])
    if resp.status_code in (200, 201):
        print("MongoDB Atlas accessList request successful", flush=True)
    else:
        print("Request problem: status code : {status_code}, content : {content}".format(status_code=resp.status_code, content=resp.content),flush=True)

import requests

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, end_date, max_results = 100):
    search_url = "https://api.twitter.com/2/tweets/search/recent"
    query_params = {'query': keyword,
                    #'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,geo.place_id',
                    'media.fields': 'duration_ms,type,url,public_metrics',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld',
                    #'sort_order': 'recency,relevancy',
                    'next_token': {}}
    return (search_url, query_params)

def create_url2(keyword, last_tweet, max_results = 100):
    search_url = "https://api.twitter.com/2/tweets/search/recent"
    query_params = {'query': keyword,
                    #'start_time': start_date,
                    'until_id': last_tweet,
                    #'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,geo.place_id',
                    'media.fields': 'duration_ms,type,url,public_metrics',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld',
                    #'sort_order': 'recency,relevancy',
                    'next_token': {}}
    return (search_url, query_params)
  

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def get_data(keyword,end_time,max_results,bearer_token):
  headers = create_headers(bearer_token)
  url = create_url(keyword, end_time, max_results)
  tweets = connect_to_endpoint(url[0], headers, url[1])
  return tweets['data']

def get_data2(keyword,last_tweet,max_results,bearer_token):
  headers = create_headers(bearer_token)
  url = create_url2(keyword, last_tweet, max_results)
  tweets = connect_to_endpoint(url[0], headers, url[1])
  return tweets['data'] 

mongod_connect = os.environ['cnx']
consumer_key        =os.environ['api_key']
consumer_secret     =os.environ['api_secret']
access_token        =os.environ['access_token']
access_token_secret =os.environ['access_secret']
bearer_token        =os.environ['bearer_token']
atlas_group_id = os.environ['project_id']
atlas_api_key_public = os.environ['public_key']
atlas_api_key_private = os.environ['private_key']

#twitter arg 
keyword = "inwi (lang:fr OR lang:ar)"
max_results = 100

keep_alive()

#cnx to mongodb
deleteAfterDate=(datetime.now()+timedelta(hours=3,minutes=5)).isoformat()
set_ip(atlas_group_id, atlas_api_key_public, atlas_api_key_private,deleteAfterDate)
ctweet=cnx(mongod_connect)
#get twitter data:
end_time= datetime.now().strftime("%Y-%m-%dT%H:")+"00:00Z"
tweets_data = get_data(keyword,end_time,max_results,bearer_token)
df = pd.json_normalize(tweets_data) 
since_id_new = tweets_data[-1]['id']      
clx={}
for i in tweets_data:
    try:
        clx[i["id"]]=i
      #print(i["created_at"])
    except Exception as e:
        print(e)
        pass
  
since_id_old = 0
while since_id_new != since_id_old:
    try:
        since_id_old = since_id_new
        tweets_data = get_data2(keyword,since_id_new,max_results,bearer_token)
        since_id_new = tweets_data[-1]['id']
        for i in tweets_data:
            try:
                clx[i["id"]]=i
                #print(i["created_at"])
            except:
                pass
    except:
        pass
    
    print('New records',len(clx))
    
    for x in clx:
        ctweet.delete_one({ "id": x })
        ctweet.insert_one(clx[x])
    
    print('collection',ctweet.count_documents({}))
    print('Done at ',datetime.now().strftime("%Y-%m-%dT%H:%M"))
    #time.sleep(10800)
  #for document in ctweet.find():
  #  clx_old[document['id']]=document
  #  del clx_old[document['id']]['_id']
  #clx_old.update(clx_new)

ImportError: cannot import name 'linux_distribution' from 'platform' (/opt/python/3.10.4/lib/python3.10/platform.py)